In [1]:
import os
import wandb
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from transformers import BlipProcessor, BlipForConditionalGeneration
from rouge import Rouge
from bert_score import score
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score
from pycocoevalcap.cider.cider import Cider

/home/nikolai/projects/album_stories/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nikolai/projects/album_stories/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/nikolai/projects/album_stories/.venv/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/nikolai/projects/album_stories/.venv/lib/python3.12/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/home/nikolai/projects/album_stories/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, data, transform=None):
        self.data_dir = data_dir
        self.data = data
        self.transform = transform
        self.images = [os.path.join(data_dir, img) for img in os.listdir(data_dir)]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image, caption = loader(image_path, self.data)
        if self.transform:
            image = self.transform(image)
        return image, caption


def loader(path, data):
    image = Image.open(path)
    caption = data.loc[data["image"] == os.path.basename(path), "caption"].values[0]
    return image, caption

In [3]:
data_dir = "datasets/sin_dataset_img/images"
csv_file = "datasets/sin_dataset_img/captions.csv"

data = pd.read_csv(csv_file)

ds_sin = CustomDataset(data_dir, data)

In [4]:
# Модель №1
wandb.init(project="child_diary", group="sin_ds", job_type="base", name="Pic2Story_0")

processor = BlipProcessor.from_pretrained("abhijit2111/Pic2Story")
model = BlipForConditionalGeneration.from_pretrained("abhijit2111/Pic2Story")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: n-hilkovich. Use `wandb login --relogin` to force relogin


In [5]:
predicted_captions = []
reference_captions = []

for image, captions in ds_sin:
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    predicted_captions.append(processor.decode(out[0], skip_special_tokens=True))
    reference_captions.append([captions])

In [6]:
predicted_dict = {i: [caption] for i, caption in enumerate(predicted_captions)}
reference_dict = {i: captions for i, captions in enumerate(reference_captions)}

cider_scorer = Cider()

cider_score, _ = cider_scorer.compute_score(reference_dict, predicted_dict)

print(f"CIDEr Score: {cider_score:.3f}")

CIDEr Score: 1.266


In [7]:
P, R, F1 = score(predicted_captions, reference_captions, lang="en", verbose=False)

BERTScore_P = P.mean().item()
BERTScore_R = R.mean().item()
BERTScore_F1 = F1.mean().item()

print(f"BERTScore Precision: {BERTScore_P:.3f}")
print(f"BERTScore Recall: {BERTScore_R:.3f}")
print(f"BERTScore F1: {BERTScore_F1:.3f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.912
BERTScore Recall: 0.924
BERTScore F1: 0.918


In [8]:
predicted_captions_tokenized = [
    word_tokenize(caption) for caption in predicted_captions
]
reference_captions_tokenized = [
    [word_tokenize(caption) for caption in ref] for ref in reference_captions
]

scores = [
    meteor_score(ref, gen)
    for ref, gen in zip(reference_captions_tokenized, predicted_captions_tokenized)
]
average_meteor = sum(scores) / len(scores)

print(f"METEOR Score: {average_meteor:.3f}")

METEOR Score: 0.495


In [9]:
ref_captions = [". ".join(ref) for ref in reference_captions]

rouge = Rouge()

scores = rouge.get_scores(predicted_captions, ref_captions, avg=True)

print("ROUGE-1 Scores:", scores["rouge-1"])
print("ROUGE-2 Scores:", scores["rouge-2"])
print("ROUGE-l Scores:", scores["rouge-l"])

ROUGE-1 Scores: {'r': 0.5586580086580086, 'p': 0.5880392156862746, 'f': 0.5612534899822796}
ROUGE-2 Scores: {'r': 0.23522727272727276, 'p': 0.26405228758169935, 'f': 0.2453215038405998}
ROUGE-l Scores: {'r': 0.4747619047619048, 'p': 0.5003431372549019, 'f': 0.47743142900504615}


In [10]:
wandb.log(
    {
        "CIDEr": cider_score,
        "BERTScore_P": BERTScore_P,
        "BERTScore_R": BERTScore_R,
        "BERTScore_F1": BERTScore_F1,
        "METEOR": average_meteor,
    }
)

wandb.finish()

print("Complite")

BERTScore_F1,▁
BERTScore_P,▁
BERTScore_R,▁
CIDEr,▁
METEOR,▁
BERTScore_F1,0.91794
BERTScore_P,0.91248
BERTScore_R,0.92372
CIDEr,1.26607
METEOR,0.49483


Complite


In [15]:
# Модель №2
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
)

In [16]:
predicted_captions = []
reference_captions = []

for image, captions in ds_sin:
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    predicted_captions.append(processor.decode(out[0], skip_special_tokens=True))
    reference_captions.append([captions])

/home/nikolai/projects/album_stories/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [17]:
predicted_dict = {i: [caption] for i, caption in enumerate(predicted_captions)}
reference_dict = {i: captions for i, captions in enumerate(reference_captions)}

cider_scorer = Cider()

cider_score, _ = cider_scorer.compute_score(reference_dict, predicted_dict)

print(f"CIDEr Score: {cider_score:.3f}")

CIDEr Score: 0.324


In [18]:
P, R, F1 = score(predicted_captions, reference_captions, lang="en", verbose=False)

print(f"BERTScore Precision: {P.mean().item():.3f}")
print(f"BERTScore Recall: {R.mean().item():.3f}")
print(f"BERTScore F1: {F1.mean().item():.3f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.951
BERTScore Recall: 0.908
BERTScore F1: 0.929


In [19]:
predicted_captions_tokenized = [
    word_tokenize(caption) for caption in predicted_captions
]
reference_captions_tokenized = [
    [word_tokenize(caption) for caption in ref] for ref in reference_captions
]

scores = [
    meteor_score(ref, gen)
    for ref, gen in zip(reference_captions_tokenized, predicted_captions_tokenized)
]
average_score = sum(scores) / len(scores)

print(f"METEOR Score: {average_score:.3f}")

METEOR Score: 0.332


In [20]:
ref_captions = [". ".join(ref) for ref in reference_captions]

rouge = Rouge()

scores = rouge.get_scores(predicted_captions, ref_captions, avg=True)

print("ROUGE-1 Scores:", scores["rouge-1"])
print("ROUGE-2 Scores:", scores["rouge-2"])
print("ROUGE-l Scores:", scores["rouge-l"])

ROUGE-1 Scores: {'r': 0.3490476190476191, 'p': 0.7047619047619048, 'f': 0.46401153954640584}
ROUGE-2 Scores: {'r': 0.17878787878787877, 'p': 0.37974025974025977, 'f': 0.24167560344166178}
ROUGE-l Scores: {'r': 0.3323809523809524, 'p': 0.6714285714285715, 'f': 0.44178931732418364}


In [21]:
# Модель №3
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-large"
)

In [22]:
predicted_captions = []
reference_captions = []

for image, captions in ds_sin:
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    predicted_captions.append(processor.decode(out[0], skip_special_tokens=True))
    reference_captions.append([captions])

In [23]:
predicted_dict = {i: [caption] for i, caption in enumerate(predicted_captions)}
reference_dict = {i: captions for i, captions in enumerate(reference_captions)}

cider_scorer = Cider()

cider_score, _ = cider_scorer.compute_score(reference_dict, predicted_dict)

print(f"CIDEr Score: {cider_score:.3f}")

CIDEr Score: 0.635


In [24]:
P, R, F1 = score(predicted_captions, reference_captions, lang="en", verbose=False)

print(f"BERTScore Precision: {P.mean().item():.3f}")
print(f"BERTScore Recall: {R.mean().item():.3f}")
print(f"BERTScore F1: {F1.mean().item():.3f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.919
BERTScore Recall: 0.908
BERTScore F1: 0.913


In [25]:
predicted_captions_tokenized = [
    word_tokenize(caption) for caption in predicted_captions
]
reference_captions_tokenized = [
    [word_tokenize(caption) for caption in ref] for ref in reference_captions
]

scores = [
    meteor_score(ref, gen)
    for ref, gen in zip(reference_captions_tokenized, predicted_captions_tokenized)
]
average_score = sum(scores) / len(scores)

print(f"METEOR Score: {average_score:.3f}")

METEOR Score: 0.365


In [26]:
ref_captions = [". ".join(ref) for ref in reference_captions]

rouge = Rouge()

scores = rouge.get_scores(predicted_captions, ref_captions, avg=True)

print("ROUGE-1 Scores:", scores["rouge-1"])
print("ROUGE-2 Scores:", scores["rouge-2"])
print("ROUGE-l Scores:", scores["rouge-l"])

ROUGE-1 Scores: {'r': 0.4447619047619047, 'p': 0.6240093240093241, 'f': 0.5105960248857122}
ROUGE-2 Scores: {'r': 0.18371212121212122, 'p': 0.3, 'f': 0.22535013627404696}
ROUGE-l Scores: {'r': 0.38567099567099566, 'p': 0.5583916083916083, 'f': 0.4494400146555332}
